In [1]:
import mysql.connector
import os
import warnings
import logging
from utils.get_all_employees import get_all_employees


HOST = os.getenv("HOST")
PASSWORD = os.getenv("PASSWORD")
USER = os.getenv("USER")
DATABASE = "sqlite_employees"

# suppress the connection warning regarding swithing to sqlalchemy
warnings.filterwarnings('ignore')

In [2]:
def insert_data_into_mysql(table_name: str, data: list, host: str, user: str, password: str, database: str, timestamp: str = None) -> None:
    """
    Insert data into a MySQL table.

    Parameters:
    -----------
    table_name : str
        Name of the table to insert data into.
    data : list
        List of tuples containing the data to insert.
    host : str
        Hostname or IP address of the MySQL server.
    user : str
        Username to connect to the MySQL server.
    password : str
        Password to connect to the MySQL server.
    database : str
        Name of the database to connect to.
    timestamp : str, optional
        Timestamp to use in the query. If not provided, uses the current timestamp.

    Raises:
    -------
    ValueError
        If any required parameters are missing or invalid.

    Returns:
    --------
    None
    """
    if not all([table_name, data, host, user, password, database]):
        raise ValueError("All required parameters must be provided.")

    # set up logging
    logging.basicConfig(filename='./logs/mysql.log', level=logging.INFO)

    # create the SQL query for the insert statement
    sql_query = f"""INSERT INTO {table_name} 
                (id, name, age, department, position, salary, created_at) 
                VALUES (%s, %s, %s, %s, %s, %s, NOW())"""

    # establish connection and create cursor
    with mysql.connector.connect(host=host, user=user, password=password, database=database) as conn:
        cursor = conn.cursor()

        try:
            # execute the query
            cursor.executemany(sql_query, [(row[0], row[1], row[2], row[3], row[4], row[5]) for row in data])

            # commit the changes to the database
            conn.commit()

            # log success message
            logging.info(f"{cursor.rowcount} records inserted successfully into the {table_name} table.")

        except mysql.connector.Error as e:
            # log error message
            logging.error(f"Failed to insert records into {table_name} table. {e}")
            raise e

        finally:
            # close cursor and connection
            cursor.close()
            conn.close()


In [3]:
insert_data_into_mysql(table_name='employees',
                       data=get_all_employees(),
                       host=HOST,
                       user=USER,
                       password=PASSWORD,
                       database=DATABASE)